https://docs.python.org/3/library/xml.etree.elementtree.html

In [1]:
from io import StringIO
import xml.etree.ElementTree as ET
import numpy as np
from copy import deepcopy

In [2]:
# Daten XML-Datei laden, Kundendaten
tree = ET.parse("/content/cnh3.XML")
root_IMPORT = tree.getroot()

# Feldgerüst XML-Datei laden
tree = ET.parse("/content/structure3.XML")
root_EXPORT = tree.getroot()


In [3]:
def is_line_straight(coords, tolerance=0.0001):
    """
    Prüft, ob eine Linie aus GPS-Koordinaten gerade oder krumm ist.

    Parameters:
        coords (list of tuples): Liste von GPS-Koordinaten [(lon1, lat1), (lon2, lat2), ...].
        tolerance (float): Maximal zulässige Abweichung von der Geraden (in Grad).

    Returns:
        bool: True, wenn die Linie gerade ist, sonst False.
        float: Maximale Abweichung von der Geraden.
    """
    if len(coords) < 3:
        # Eine Linie mit weniger als 3 Punkten ist immer "gerade"
        return True, 0.0

    # Extrahiere Start- und Endpunkte
    start = np.array(coords[0])
    end = np.array(coords[-1])

    # Richtungsvektor der idealen Linie
    line_vector = end - start

    # Normiere den Richtungsvektor
    line_vector_normalized = line_vector / np.linalg.norm(line_vector) # linalg bedeutet Lineare Algebra

    max_deviation = 0.0 #maximale Abweichung resetten

    # Prüfe jeden Punkt auf Abweichung
    for point in coords[1:-1]:  # Ignoriere Start- und Endpunkte
        point_vector = np.array(point) - start #Array aus dem Punkt erzeugen, die Richtung vom Start aus gesehen berechnen

        # Projektion des Punktes auf den Richtungsvektor
        projected_length = np.dot(point_vector, line_vector_normalized) #  Skalarprodukt bestimmen zwischen Punktvektor und Vektor der Linie
        projected_point = start + projected_length * line_vector_normalized #Projektion bestimmen

        # Abweichung des Punktes von der Linie
        deviation = np.linalg.norm(np.array(point) - projected_point)# berechnet die euklidische Distanz zwischen dem Punkt und seinem projizierten Punkt. der Senkrecht , kürzeste Abstand
        max_deviation = max(max_deviation, deviation) #kontrolliert ob sich die Abweichung gesteigert hat

    # Linie gilt als gerade, wenn maximale Abweichung <= Toleranz
    return max_deviation <= tolerance, max_deviation

# Feldgerüst bauen

In [4]:
Feldgerüst=root_EXPORT.find("PFD") #Feldgerüst

## Iterationsvariablen anlegen

In [5]:
PFD_ID=0
GGP_ID=0
GPN_ID=0
PLN_ID=0
LSG_ID=0

## GPN finden und entfernen

In [6]:
parent_GGP = Feldgerüst.find("GGP")#erstes GGP(Parent finden) finden
child_GPN = parent_GGP.find("GPN")#erstes GPN(Parent finden) finden, man kann quasi keinen Parent überspringen
parent_GGP.remove(child_GPN)#löschen

child_LSG = Feldgerüst.find("LSG")
Feldgerüst.remove(child_LSG)#löschen

## Feld name ändern

In [7]:
Feldname = Feldgerüst.set("C","Acker222")# Name ändern

Feldgerüst.set("A",(f"PFD-{PFD_ID}"))#PFD Nummer
PFD_ID+=1

In [8]:
Bauer=  root_EXPORT.find("FRM")# Attribute verändern

Bauer.attrib

Bauer.set("B","Bauer_Joe")

del Feldgerüst.attrib["C"] # Attribut löschen


In [9]:
pfd_export_list=[] # Feld Exportliste erstellen

# Felder extrahieren und aufbereiten

        XPath-Ausdruck	Bedeutung

    * .	Aktuelles Element
    * .//PLN	Alle < PLN>-Elemente im aktuellen Kontext
    * .//PLN/LSG	Alle < LSG>-Elemente, die Kinder von < PLN> sind
    * .//LSG[@A="1"]	Alle < LSG>-Elemente mit dem Attribut A="1"
    * .//PNT[@C="52.763252327"]	Alle < PNT>-Elemente mit dem Attribut C="52763252327"
    * .//LSG[1]	Das erste < LSG>-Element
    * .//PNT[last()]	Das letzte < PNT>-Element
    * .//PLN/LSG/PNT[@A="10"]	Alle < PNT>-Elemente unter < LSG> unter < PLN> mit Attribut A="10"
    

# hier Iteration FOR-Schleife!

In [10]:
Feldgerüst_c = deepcopy(Feldgerüst)

In [11]:
print(ET.tostring(Feldgerüst_c, encoding="unicode"))

<PFD F="FRM1" D="0" A="PFD-0" E="CTR1">
		<PLN A="1" B="Gen4-Vermessung">
		</PLN>
		<GGP A="GGP-0001" B="pikeneer_AB">
			</GGP>
		</PFD>
	


In [12]:
pfd_elements = root_IMPORT.findall("PFD") # findall() ist finde alle; find() ist finde ersten

In [13]:
len(pfd_elements)

9

In [14]:
def AGCO(pfd_elements, Feldgerüst):
  PFD_ID=0
  GGP_ID=0
  GPN_ID=0
  PLN_ID=0
  LSG_ID=0
  for pfd_n in pfd_elements[:30]:

    Feldgerüst_c = deepcopy(Feldgerüst)
    parent_GGP_c = Feldgerüst_c.find("GGP")
    parent_GGP_c.set("A",(f"GGP-{GGP_ID}"))
    GGP_ID+=1

    #_________________
    pln_n = pfd_n.find(".//PLN")

    Name_Feld=pfd_n.get("C")
    Feldgerüst_c.set("C",Name_Feld)
    Feldgerüst_c.set("A",(f"PFD-{PFD_ID}"))
    PFD_ID+=1
    #_________________
    pln_n = pfd_n.find(".//PLN/LSG") # über xpath, findet die erste Feldgrenze

    print(ET.tostring(pfd_n, encoding="unicode"))

    if pln_n is None:
        continue

    Punkte_Feld = pln_n.findall("PNT")
    for Punkt in Punkte_Feld:
      Punkt.set("A","9")
      try:
          del Punkt.attrib["E"]  # E löschen, falls vorhanden
      except KeyError:
          pass  # Wenn "E" nicht vorhanden ist, ignoriere den Fehler

    #print(ET.tostring(pln_n, encoding="unicode"))
    #_________________

    spuren=pfd_n.findall(".//GGP/GPN") #Bei AGCO hat jede Spur GGP und GPN

    spuren_ex= [] #Spuren für den Export, nur die geraden Geraden

    pln_child_c = Feldgerüst_c.find("PLN")#Feld durchnummerieren
    pln_child_c.set("A","1") # das musste bei cnh angepasst werden
    #PLN_ID+=1

    for spur in spuren:
      Punkte=spur.findall(".//LSG/PNT")#Die Punkte extrahieren
      #Name_spur = spur.get("B")
      spur.set("A",(f"GPN-{GPN_ID}"))
      GPN_ID+=1

      Spur_Test = []

      for uu in Punkte:
        longi = uu.get("C")
        lati = uu.get("D")
        Spur_Test.append((float(longi),float(lati)))

      gerade,Abweichung =is_line_straight(Spur_Test,0.00015)

      for uu in Punkte:
        try:
            del uu.attrib["E"]  # E löschen, falls vorhanden
        except KeyError:
            pass  # Wenn "E" nicht vorhanden ist, ignoriere den Fehler
        uu.set("A","2")   # A ist 2, einfacher Punkt

      if gerade==True:
        spuren_ex.append(spur)

      #print(f"{Name_spur} ist gerade {gerade} mit der Abweichung {Abweichung} ")
  ## Spuren einkürzen
    for spur in spuren_ex: #Spuren auf zwei Punkte eindampfen
      # Ziel-LSG finden
        Name_spur = spur.get("B")
        lsg = spur.find(".//LSG")
        if Name_spur is not None:
          lsg.set("B",Name_spur)
        if lsg is None:
          continue

        # Alle PNT-Knoten ermitteln
        pnt_elements = lsg.findall("PNT")
        if len(pnt_elements)>2:
          for pnt in pnt_elements[1:-1]:  # Alle außer das erste und letzte
            lsg.remove(pnt)
  ## Spuren in Form bringen GPN und LSG
    GPN_sort=spuren_ex
    LSG_sort=[]
    for spur in GPN_sort:
      child_LSG=spur.find("LSG")
      spur.set("C","1")
      LSG_sort.append(child_LSG)
      spur.remove(child_LSG)

  ## Feld in Feldgerüst einsetzen
    pln_child_c.insert(0, pln_n)

 ## GPNs in Feldgerüst einsetzen
    for i, spur in enumerate(GPN_sort):
      parent_GGP_c.insert(i,spur)

  ## LSGs in Feldgerüst einsetzen
    # Füge die neuen Elemente hinter <GGP> ein
    for i, LSG in enumerate(LSG_sort):
        Feldgerüst_c.insert(list(Feldgerüst_c).index(parent_GGP_c) + 1 + i, LSG)

  ## an die Feldliste hängen
    pfd_export_list.append(Feldgerüst_c)
    #Test_2 = ET.tostring(Feldgerüst_c, encoding="unicode")
    #pln_child.clear()
    LSG_sort.clear()

    return pfd_export_list

In [15]:
#Liste_AGCO=AGCO(pfd_elements,Feldgerüst)

In [16]:
#def CNH(pfd_elements, Feldgerüst):

PFD_ID=0
GGP_ID=0
GPN_ID=0
PLN_ID=0
LSG_ID=0
for pfd_n in pfd_elements:

  Feldgerüst_c = deepcopy(Feldgerüst)
  parent_GGP_c = Feldgerüst_c.find("GGP")
  parent_GGP_c.set("A",(f"GGP-{GGP_ID}"))
  GGP_ID+=1

  #print(ET.tostring(pfd_n, encoding="unicode"))
  #_________________


  Name_Feld=pfd_n.get("C")
  Feldgerüst_c.set("C",Name_Feld)
  Feldgerüst_c.set("A",(f"PFD-{PFD_ID}"))
  PFD_ID+=1
  #_________________
  pln_n = pfd_n.find('.//GGP/GPN[@B="Feld 1"]/LSG')

  #print(ET.tostring(pln_n, encoding="unicode"))

  if pln_n is not None:
      #continue
    #print(ET.tostring(pln_n, encoding="unicode")) hier Problem
    pln_n.set("A","1")

    Punkte_Feld = pln_n.findall("PNT")
    for Punkt in Punkte_Feld:
      Punkt.set("A","9")
      try:
          del Punkt.attrib["E"]  # E löschen, falls vorhanden
      except KeyError:
          pass  # Wenn "E" nicht vorhanden ist, ignoriere den Fehler

      ## Feld in Feldgerüst einsetzen
    Feldgerüst_c.find("PLN").insert(0, pln_n)
    Feldgerüst_c.find("PLN").set("A","1")
  else:
    pln_c=Feldgerüst_c.find("PLN")
    Feldgerüst_c.remove(pln_c)


  #print(ET.tostring(pln_n, encoding="unicode"))
  #_________________

  spuren=pfd_n.findall('.//GGP/GPN[@C="1"]') #hier war eine Fehler mit Find und findall

  spuren_ex= [] #Spuren für den Export, nur die geraden Geraden

  #PLN_ID+=1

  for spur in spuren:
    Punkte=spur.findall(".//LSG/PNT")#Die Punkte extrahieren
    #Name_spur = spur.get("B")
    spur.set("A",(f"GPN-{GPN_ID}"))
    GPN_ID+=1

    Spur_Test = []

    for uu in Punkte:
      longi = uu.get("C")
      lati = uu.get("D")
      Spur_Test.append((float(longi),float(lati)))

    gerade,Abweichung =is_line_straight(Spur_Test,0.00015)

    for uu in Punkte:
      try:
          del uu.attrib["E"]  # E löschen, falls vorhanden
      except KeyError:
          pass  # Wenn "E" nicht vorhanden ist, ignoriere den Fehler
      uu.set("A","2")   # A ist 2, einfacher Punkt

    if gerade==True:
      spuren_ex.append(spur)

  #print(ET.tostring(spuren_ex[0], encoding="unicode"))

    #print(f"{Name_spur} ist gerade {gerade} mit der Abweichung {Abweichung} ")
## Spuren einkürzen
  for spur in spuren_ex: #Spuren auf zwei Punkte eindampfen
    # Ziel-LSG finden
      Name_spur = spur.get("B")
      lsg = spur.find(".//LSG")
      if Name_spur is not None:
        lsg.set("B",Name_spur)
      if lsg is None:
        continue

      # Alle PNT-Knoten ermitteln
      pnt_elements = lsg.findall("PNT")
      if len(pnt_elements)>2:
        for pnt in pnt_elements[1:-1]:  # Alle außer das erste und letzte
          lsg.remove(pnt)
## Spuren in Form bringen GPN und LSG
  GPN_sort=spuren_ex
  LSG_sort=[]

  for spur in GPN_sort:
    child_LSG=spur.find("LSG")
    spur.set("C","1")
    LSG_sort.append(child_LSG)
    #print(ET.tostring(child_LSG, encoding="unicode"))
    spur.remove(child_LSG)



## GPNs in Feldgerüst einsetzen
  for i, spur in enumerate(GPN_sort):
    parent_GGP_c.insert(i,spur)

## LSGs in Feldgerüst einsetzen
  # Füge die neuen Elemente hinter <GGP> ein
  for i, LSG in enumerate(LSG_sort):
      Feldgerüst_c.insert(list(Feldgerüst_c).index(parent_GGP_c) + 1 + i, LSG)


## an die Feldliste hängen
  pfd_export_list.append(Feldgerüst_c)
  #Test_2 = ET.tostring(Feldgerüst_c, encoding="unicode")
  #pln_child.clear()
  LSG_sort.clear()

    #return pfd_export_list

In [17]:
#Liste_cnh=CNH(pfd_elements,Feldgerüst)

In [18]:
#pfd_export_list=Liste_cnh

#Exportdatei

In [19]:
print(ET.tostring(root_EXPORT, encoding="unicode"))

<ISO11783_TaskData ManagementSoftwareManufacturer="AGCO Field Logistics" TaskControllerManufacturer="John Deere" VersionMinor="2" ManagementSoftwareVersion="1.10.2" VersionMajor="4" DataTransferOrigin="2" TaskControllerVersion="3">
	<PGP A="PGP1" B="CropType" C="2" />
	<CTR A="CTR1" B="Kunde" />
	<FRM A="FRM1" B="Bauer_Joe" I="CTR1" />
	<PFD F="FRM1" D="0" A="PFD-0" E="CTR1">
		<PLN A="1" B="Gen4-Vermessung">
		</PLN>
		<GGP A="GGP-0001" B="pikeneer_AB">
			</GGP>
		</PFD>
	<CTP A="CTP-1" B="CropType" />
</ISO11783_TaskData>


In [20]:
root_EXPORT.find("PFD").clear()

In [21]:
print(ET.tostring(root_EXPORT, encoding="unicode"))

<ISO11783_TaskData ManagementSoftwareManufacturer="AGCO Field Logistics" TaskControllerManufacturer="John Deere" VersionMinor="2" ManagementSoftwareVersion="1.10.2" VersionMajor="4" DataTransferOrigin="2" TaskControllerVersion="3">
	<PGP A="PGP1" B="CropType" C="2" />
	<CTR A="CTR1" B="Kunde" />
	<FRM A="FRM1" B="Bauer_Joe" I="CTR1" />
	<PFD /><CTP A="CTP-1" B="CropType" />
</ISO11783_TaskData>


In [22]:
frm=root_EXPORT.find("FRM")
# Zielindex finden
index_target = list(root_EXPORT).index(frm)

# Elemente direkt hinter "Target" einfügen
for i, pfds in enumerate(pfd_export_list):
    root_EXPORT.insert(index_target + 1 + i, pfds)


In [23]:
print(ET.tostring(root_EXPORT, encoding="unicode"))

<ISO11783_TaskData ManagementSoftwareManufacturer="AGCO Field Logistics" TaskControllerManufacturer="John Deere" VersionMinor="2" ManagementSoftwareVersion="1.10.2" VersionMajor="4" DataTransferOrigin="2" TaskControllerVersion="3">
	<PGP A="PGP1" B="CropType" C="2" />
	<CTR A="CTR1" B="Kunde" />
	<FRM A="FRM1" B="Bauer_Joe" I="CTR1" />
	<PFD F="FRM1" D="0" A="PFD-0" E="CTR1" C="1">
		<GGP A="GGP-0" B="pikeneer_AB">
			<GPN A="GPN-0" B="AB-Gerade 10" C="1" E="1" F="1" G="0" I="0">
        </GPN>
    <GPN A="GPN-1" B="AB-Gerade 3" C="1" E="1" F="1" G="0" I="0">
        </GPN>
    <GPN A="GPN-2" B="AB-Gerade 8" C="1" E="1" F="1" G="0" I="0">
        </GPN>
    <GPN A="GPN-3" B="AB-Gerade 7" C="1" E="1" F="1" G="0" I="0">
        </GPN>
    <GPN A="GPN-4" B="AB-Gerade 2" C="1" E="1" F="1" G="0" I="0">
        </GPN>
    <GPN A="GPN-5" B="AB-Gerade 6" C="1" E="1" F="1" G="0" I="0">
        </GPN>
    <GPN A="GPN-6" B="AB-Gerade 9" C="1" E="1" F="1" G="0" I="0">
        </GPN>
    <GPN A="GP

# Dateiausgabe

In [24]:

tree_out = ET.ElementTree(root_EXPORT)
# Datei speichern
with open("TASKDATA.xml", "wb") as file:
    tree_out.write(file, encoding="utf-8", xml_declaration=True)